In [12]:
import time
import json
%reload_ext e2emlstorlets.tools.ipython

In [3]:
import os
os.environ['OS_AUTH_VERSION'] = '3'
os.environ['OS_AUTH_URL'] = 'http://127.0.0.1:5000/v3'
os.environ['OS_USERNAME'] = 'tester'
os.environ['OS_PASSWORD'] = 'testing'
os.environ['OS_USER_DOMAIN_NAME'] = 'default'
os.environ['OS_PROJECT_DOMAIN_NAME'] = 'default'
os.environ['OS_PROJECT_NAME'] = 'test'

In [4]:
%%storletapp extract_face.ExtractFace
import cv2
import numpy as np

def detect(im):
    mat=cv2.imdecode(im, cv2.IMREAD_GRAYSCALE)
    cascade = cv2.CascadeClassifier("/usr/local/share/OpenCV/haarcascades/haarcascade_frontalface_alt.xml")
    rects = cascade.detectMultiScale(mat)

    if len(rects) == 0:
        return [], mat
    rects[:, 2:] += rects[:, :2]
    rect = rects[0]
    return mat, rect

def crop(img, rect):
    h = rect[3]-rect[1]
    w = rect[2]-rect[0]
    x = rect[0]
    y = rect[1]
    return img[y:y+h, x:x+w]

class ExtractFace(object):
    def __init__(self, logger):
        self.logger = logger

    def __call__(self, in_files, out_files, params):
        metadata = in_files[0].get_metadata()
        out_files[0].set_metadata(metadata)

        # Read the image
        img_str = ''
        while True:
            buf = in_files[0].read(1024)
            if not buf:
                break
            img_str += buf
        img_nparr = np.fromstring(img_str, np.uint8)
        
        # Detect face
        mat, rect = detect(img_nparr)

        # Crop the face and decrease resolution
        face = crop(mat, rect)
        small_face = cv2.resize(face, (30,30))
        
        # Write result
        retval, small_face_buf = cv2.imencode('.jpg', small_face)
        out_files[0].write(small_face_buf)
        
        in_files[0].close()
        out_files[0].close()
        self.logger.debug('Done\n')

Upload storlets succeeded /storlet/extract_face.py
Example command `swift download <container> <object> -H X-Run-Storlet:extract_face.py`


In [7]:
# Iterate over all puctures, and extract faces
start_time = time.time()
%list_container -i tr -o obj_list
for obj in obj_list:
    input_path=os.path.join('/tr',obj)
    output_path=os.path.join('/str',obj)
    %copy --storlet extract_face.py --input path:$input_path --output path:$output_path -o result
extract_faces_time = time.time() - start_time
print('Done extracting faces in %d seconds' % round(extract_faces_time))

Done extracting faces in 16


In [28]:
# Train the model with all faces and name to id dictionary
start_time = time.time()
name_to_id = {'bibi': 1, 'merkel': 2, 'obama': 3, 'trump': 4}
params = {'name_to_id': json.dumps(name_to_id)}
input_path = os.path.join('path:str/', obj_list[0])
extra_resources = ','.join([ '/str/%s' % obj_name for obj_name in obj_list[1:]])
output_path = 'path:/trained/model'
%copy --storlet train_model.py --input $input_path --output $output_path --extra $extra_resources -i params -o result
train_model_time = time.time() - start_time
print('Model training done in %d seconds' % round(train_model_time))

201


In [29]:
# Swap the face
start_time = time.time()
input_path = 'path:/video/eran_mov.avi'
output_path = 'path:/video/eran_swapped_mov.avi'
extra_resources = '/te/bibi_test1.jpeg'
%copy --storlet video_swap_face.py --input $input_path --output $output_path --extra $extra_resources -o result
swap_faces_time = time.time() - start_time
print('Swap facing done in %d seconds' % round(swap_faces_time))

201


In [30]:
# Add face recognition tag
start_time = time.time()
input_path = 'path:/video/eran_swapped_mov.avi'
output_path = 'path:/video/tagged_mov.avi'
extra_resources = '/trained/model'
%copy --storlet video_recognize_face.py --input $input_path --output $output_path --extra $extra_resources -o result
recognize_face_time = time.time() - start_time
print('Recognize face done in %d seconds' % round(recognize_face_time))

201


In [13]:
%play_video -c video -v eran_mov.avi